<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Instantiate-CPPN" data-toc-modified-id="Instantiate-CPPN-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Instantiate CPPN</a></span></li><li><span><a href="#Generate-Image" data-toc-modified-id="Generate-Image-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Generate Image</a></span><ul class="toc-item"><li><span><a href="#Export" data-toc-modified-id="Export-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Export</a></span></li></ul></li><li><span><a href="#Animation" data-toc-modified-id="Animation-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Animation</a></span><ul class="toc-item"><li><span><a href="#Marching-Cubes" data-toc-modified-id="Marching-Cubes-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Marching Cubes</a></span></li></ul></li><li><span><a href="#Parameters-Grid-Search" data-toc-modified-id="Parameters-Grid-Search-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Parameters Grid Search</a></span></li></ul></div>

In [ ]:
import numpy as np
import yaml
import os
import cv2
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import animation
from datetime import datetime

from pathlib import Path

plt.rcParams['animation.ffmpeg_path'] = str(Path.home() / "anaconda3/envs/image-processing/bin/ffmpeg")

%matplotlib notebook

%load_ext autoreload
%autoreload 2

from CPPN import CPPN
from ds_utils.voxel_utils import get_sphere_mask
from ds_utils.video_utils import generate_video

# Instantiate CPPN

In [ ]:
res_path = Path.home() / 'Documents/generated_data/cppn'

In [ ]:
extra_funs= {
    'base': lambda x,y,z: x*y*z,
    'cos_sin': lambda x,y,z: np.cos(x)*np.sin(y)*np.sin(z),
    'cube': lambda x,y,z: x**3 + 3*y - y**3 -3*x + z**2 -z,
    'rand': lambda x,y,z: np.sqrt(x*x+y*y+z*z) + (x*x) + np.tan(y) + 3*z,
}

In [ ]:
# load config file
with open('cppn_config.yaml', 'r') as f:
    model_config = yaml.load(f, Loader=yaml.FullLoader)
model_config = model_config['base_bw']
model_config

In [ ]:
# OPTIONALLY customize model config
model_config['nb_hidden_layers'] = 8
#model_config['kernel_init_stddev'] = 1.
model_config['kernel_init_mean'] = 0.
model_config['nb_channels'] = 1

#model_config['inner_architecture_key'] = 'residual'

In [ ]:
# init model
batch_size = 5

img_width = img_height = 100
img_depth = 1
img_size = (img_width, img_height)

cppn = CPPN(batch_size=batch_size, img_width=img_width, img_height=img_height, img_depth=img_depth,
            **model_config)

In [ ]:
cppn.model.summary()

# Generate Image

In [ ]:
x, y, z, r, e = cppn.get_data(extra_fun=extra_funs['base'])
latent = cppn.get_latent()

In [ ]:
result_imgs = cppn.generate_imgs(x, y, z, r, e, latent)
# as the results are 3D, we select only one slice through the 3rd dimension
plt.imshow(result_imgs[0, :, :, 0], cmap='gray')

## Export

In [ ]:
# export results as numpy
np.save(str(res_path / 'bw_3d_100_sphere.npy'), result_imgs[0])

In [ ]:
# test load results
np.load(str(res_path / 'numpy_exports/bw_1080.npy')).shape

In [ ]:
# export as images
for i, img in enumerate(result_imgs): 
    plt.imsave(str(res_path / f'numpy_exports/sample_{i}.png'), img, cmap='gray')

# Animation

In [ ]:
def animate_cppn(cppn, nb_frames: int, add_val: float, animate_data=False):
    cppn_snapshot = []
    
    latent = cppn.get_latent()
    x, y, z, r, e = cppn.get_data(scale=data_config['scale'], 
                               translation=data_config['translation'], 
                               rotation=data_config['rotation'], 
                               extra_fun=extra_funs[data_config['extra_fun']])

    for i in range(nb_frames):
        if i%10 == 0:
            print(i)
        latent_idx = int(i/(int(nb_frames/model_config['latent_dim'])))
        #latent[0][latent_idx] += add_val
        latent[0] += add_val
        
        if animate_data:
            x, y, z, r, e = cppn.get_data(scale=data_config['scale']+i*data_config['scale_speed'], 
                                       translation=data_config['translation']+i*data_config['translation_speed'], 
                                       rotation=data_config['rotation']+i*data_config['rotation_speed'], 
                                       extra_fun=extra_funs[data_config['extra_fun']])

        cppn_snapshot.append(cppn.generate_imgs(x, y, z, r, e, latent)[0])

    cppn_snapshot = np.array(cppn_snapshot)
    return cppn_snapshot

In [ ]:
save_anim = False    # whether to save animation to file
animate_data = True  # whether to retrieve new input data at each animation frame
FRAMES = 100
batch_size = 1
img_width = img_height = img_depth = 50
img_size = (img_width, img_height)

# Init model and data
with open('cppn_config.yaml', 'r') as f:
    model_config = yaml.load(f, Loader=yaml.FullLoader)
data_config = model_config['test_config']
model_config = model_config['base_bw']
cppn = CPPN(batch_size=batch_size, img_width=img_width, img_height=img_height, img_depth=img_depth,
            **model_config)

cppn_snapshot = []
latent_max_val = 1
#latent = np.zeros((1, model_config['z_dim']))-latent_max_val
add_val = (latent_max_val*1)/FRAMES
#add_val = 1.0

cppn_snapshot = animate_cppn(cppn, FRAMES, add_val)

In [ ]:
np.save(res_path / f'test_3D_anim.npy', cppn_snapshot)

In [ ]:
# Setup plot
dpi = 100
if save_anim:
    fig, ax = plt.subplots(dpi=dpi, figsize=(img_width/dpi, img_height/dpi))
else:
    fig, ax = plt.subplots(dpi=dpi, figsize=(5, 5))
plt.axis('off')

def animate(i, ax, cppn_snapshot):
    ax.imshow(cppn_snapshot[i], cmap='gray')

# Animate
ani = animation.FuncAnimation(fig, animate, frames=FRAMES, interval=100,
                              fargs=[ax, cppn_snapshot])
if save_anim:
    ani.save(str(res_path / 'tests' / 'anim_{}.mp4'.format(datetime.strftime(datetime.now(), "%Y-%m-%d_%H-%M"))), 
             animation.FFMpegFileWriter(fps=30))

## Marching Cubes

In [ ]:
import mcubes

In [ ]:
vertices, triangles = mcubes.marching_cubes(cppn_snapshot[0], 0.2)

#mcubes.export_obj(vertices, triangles, res_path / 'test.obj')

In [ ]:
triangles

# Parameters Grid Search

In [ ]:
from sklearn.model_selection import ParameterGrid

In [ ]:
param_grid = {
                #'inner_architecture_key': ['base', 'residual', 'softplus'],
                'kernel_init_stddev': np.linspace(0.7, 4., num=4),
                #'scale': np.linspace(-2., 2., num=5),
                #'translation': np.linspace(-4., 4., num=3),
                #'rotation': np.linspace(1, 360, num=4),
                #'nb_hidden_layers': np.arange(3, 7, 2),
                #'z_dim': [8, 16, 32, 64],
                'hidden_dim': [8, 16],
                'extra_fun': ['base', 'cos_sin', 'cube', 'rand']
             }
grid = ParameterGrid(param_grid)

# Init model and data
with open('cppn_config.yaml', 'r') as f:
    model_config = yaml.load(f, Loader=yaml.FullLoader)
data_config = model_config['test_config']
model_config = model_config['base_bw']

nb_frames = 100
batch_size = 1
img_height = img_width = 100
img_depth = img_height

out_dir = res_path / 'gs' / f'{img_width}x3_{nb_frames}frames'
out_dir.mkdir(parents=True, exist_ok=False) 

sphere_mask = get_sphere_mask((img_height,img_width,img_depth), (img_height//2)-1)
    
with open(str(out_dir / 'logs.txt'), 'w+') as f:
    for run, params in enumerate(grid):
        print("Params {}: {}".format(run, params))
        
        current_config = model_config.copy()
        current_data_config = data_config.copy()
        current_config.update(params)
        current_data_config.update(params)
        
        cppn = CPPN(batch_size=batch_size, img_width=img_width, img_height=img_height, img_depth=img_depth,
                **current_config)
        
        latent_max_val = 1
        #latent = np.zeros((1, model_config['z_dim']))-latent_max_val
        add_val = (latent_max_val*1)/nb_frames
        #add_val = 1.0

        cppn_snapshot = animate_cppn(cppn, nb_frames, add_val)
        
        # write out config
        f.write(str(current_config) + str(current_data_config) + '\n')

        # write out numpy 4D tensor
        np.save(out_dir / f'run_{run}.npy', np.array(cppn_snapshot, dtype=np.float16))
        
        

        # write out as sliced videos
        run_out_path = out_dir / f'vid_run_{run:03}'
        run_out_path.mkdir(exist_ok=False, parents=True)
        for z_coord in range(img_depth):
            generate_video(str(run_out_path / f"{z_coord}.mp4"),
                           (img_width, img_height),
                           frame_gen_fun=lambda i: cv2.normalize(cppn_snapshot[i, :, :, z_coord], None, 255, 0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U),
                           nb_frames=len(cppn_snapshot), is_color=False, disable_tqdm=True)
            
                
        #sphere masked
        cppn_snapshot = cppn_snapshot * sphere_mask[ np.newaxis, :, :, :]
        np.save(out_dir / f'run_{run}_sphere.npy', np.array(cppn_snapshot, dtype=np.float16))
        
        #imgs = cppn.generate_imgs(x, y, r, e, z)
        #for j, img in enumerate(imgs): 
        #    plt.imsave(str(out_dir / f'sample_{j}.png'), img, cmap='gray')